In [1]:
from dataprocessing.processors.refined_ingestion import TrackingDataRefinedProcess, TimetableRefinedProcess, BusStopRefinedProcess, LineRefinedProcess
from dataprocessing.processors.trust_ingestion import TrustProcessing

from pyspark.sql import functions as F, types as T
from pyspark.sql.window import Window

In [2]:
# from dataprocessing.processors.sparketl import ETLSpark

# etl_spark = ETLSpark()

# df = etl_spark.sqlContext.read.parquet("/data/trusted/vehicles").withColumn("hour", F.hour(F.col("event_timestamp"))).sort(F.asc("event_timestamp"))

In [13]:
%%time
TrustProcessing("2019-05")()

FILE: Column<b'input_file_name()'>
FILE: Column<b'input_file_name()'>
FILE: Column<b'input_file_name()'>
CPU times: user 42.7 ms, sys: 13.6 ms, total: 56.3 ms
Wall time: 37.6 s


In [15]:
%%time
LineRefinedProcess(2019,5,2)()
TimetableRefinedProcess(2019,5,2)()
BusStopRefinedProcess(2019,5,2)()
TrackingDataRefinedProcess(2019,5,2)()

CPU times: user 1.25 s, sys: 334 ms, total: 1.58 s
Wall time: 4min 55s


## AREA 51

In [2]:
trd = TrackingDataRefinedProcess(2019,5,2)

vehicles = trd.compute_metrics()

stop_events = trd.stop_events(vehicles)

In [3]:
stop_events.filter("vehicle = 'BN997'").sort(F.asc("stop_timestamp")).show(50)

+---------+--------+-------+-------------+-------------------+----------+----------+------------+----+-----+---+----+
|line_code|line_way|vehicle|moving_status|     stop_timestamp|  latitude| longitude|avg_velocity|year|month|day|hour|
+---------+--------+-------+-------------+-------------------+----------+----------+------------+----+-----+---+----+
|      001| Horario|  BN997|      STOPPED|2019-05-02 06:21:00|-25.427685|-49.264708|        9.83|2019|    5|  2|   6|
|      001| Horario|  BN997|      STOPPED|2019-05-02 06:23:24|-25.432431|-49.262705|         6.0|2019|    5|  2|   6|
|      001| Horario|  BN997|      STOPPED|2019-05-02 06:24:07|-25.433933| -49.26203|        7.18|2019|    5|  2|   6|
|      001| Horario|  BN997|      STOPPED|2019-05-02 06:25:13| -25.43442|-49.262871|         7.2|2019|    5|  2|   6|
|      001| Horario|  BN997|      STOPPED|2019-05-02 06:27:33|-25.436813|-49.268886|        7.82|2019|    5|  2|   6|
|      001| Horario|  BN997|      STOPPED|2019-05-02 06:

In [4]:
bus_event_edges = trd.event_edges(vehicles)

events = trd.process_events(stop_events, bus_event_edges)

In [5]:
events.filter("vehicle = 'BN997'").sort(F.asc("event_timestamp")).show(50)

+---------+--------+-------+-------------+-------------------+----------+----------+-------------------+----+-----+---+----+-------------------+-----------------+
|line_code|line_way|vehicle|moving_status|    event_timestamp|  latitude| longitude|       avg_velocity|year|month|day|hour|     last_timestamp|delta_time_in_sec|
+---------+--------+-------+-------------+-------------------+----------+----------+-------------------+----+-----+---+----+-------------------+-----------------+
|      001| Horario|  BN997|      STOPPED|2019-05-02 06:21:00|-25.427685|-49.264708|               9.83|2019|    5|  2|   6|                  0|             null|
|      001| Horario|  BN997|       MOVING|2019-05-02 06:21:06|-25.428013| -49.26449|              29.15|2019|    5|  2|   6|2019-05-02 06:21:00|                6|
|      001| Horario|  BN997|      STOPPED|2019-05-02 06:21:27|  -25.4283|-49.264411|               5.64|2019|    5|  2|   6|2019-05-02 06:21:06|               21|
|      001| Horario|  